In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping
import matplotlib.dates as mdates

# Folder paths
base_folder = './output/input_data_characterization/plots/'
input_file = base_folder + 'inputdata/revised_realistic_synthetic_data_with_improvements.csv'
output_folder = base_folder + 'outputdata/LSTM/'
model_files_folder = os.path.join(output_folder, 'model_files/')
predictions_folder = os.path.join(output_folder, 'predictions/')
plots_folder = os.path.join(output_folder, 'plots/')

# Ensure folders exist or create them
os.makedirs(model_files_folder, exist_ok=True)
os.makedirs(predictions_folder, exist_ok=True)
os.makedirs(plots_folder, exist_ok=True)

# Load and preprocess data
data = pd.read_csv(input_file)
data['Month'] = pd.to_datetime(data['Month'])
data.set_index('Month', inplace=True)

# Ensure the data frequency is monthly and covers the date range from 2024 to 2029
data = data.asfreq('MS')  # Monthly Start frequency

# Select relevant columns (Demand and Supply)
demand_data = data['Demand'].values.reshape(-1, 1)
supply_data = data['Supply'].values.reshape(-1, 1)

# Scaling the data using MinMaxScaler
scaler_demand = MinMaxScaler(feature_range=(0, 1))
scaler_supply = MinMaxScaler(feature_range=(0, 1))

# Scale the full range of data (2024–2029)
demand_scaled = scaler_demand.fit_transform(demand_data)
supply_scaled = scaler_supply.fit_transform(supply_data)

# Helper function to create sequences for LSTM input
def create_sequences(data, time_step=1):
    x_data, y_data = [], []
    for i in range(len(data) - time_step):
        x_data.append(data[i:(i + time_step), 0])
        y_data.append(data[i + time_step, 0])
    return np.array(x_data), np.array(y_data)

# Time step of 3 months
time_step = 3
X_demand, y_demand = create_sequences(demand_scaled, time_step)
X_supply, y_supply = create_sequences(supply_scaled, time_step)

# Reshape input to be [samples, time steps, features] for LSTM
X_demand = X_demand.reshape((X_demand.shape[0], X_demand.shape[1], 1))
X_supply = X_supply.reshape((X_supply.shape[0], X_supply.shape[1], 1))

# Create the LSTM models
model_demand = Sequential()
model_demand.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
model_demand.add(LSTM(50, return_sequences=False))
model_demand.add(Dense(25))
model_demand.add(Dense(1))
model_demand.compile(optimizer='adam', loss='mean_squared_error')

model_supply = Sequential()
model_supply.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
model_supply.add(LSTM(50, return_sequences=False))
model_supply.add(Dense(25))
model_supply.add(Dense(1))
model_supply.compile(optimizer='adam', loss='mean_squared_error')

# Train the LSTM models
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Split the data for training and validation
train_size = int(len(X_demand) * 0.8)
X_train_demand, X_test_demand = X_demand[:train_size], X_demand[train_size:]
y_train_demand, y_test_demand = y_demand[:train_size], y_demand[train_size:]

X_train_supply, X_test_supply = X_supply[:train_size], X_supply[train_size:]
y_train_supply, y_test_supply = y_supply[:train_size], y_supply[train_size:]

model_demand.fit(X_train_demand, y_train_demand, validation_data=(X_test_demand, y_test_demand), epochs=50, batch_size=32, verbose=1, callbacks=[early_stop])
model_supply.fit(X_train_supply, y_train_supply, validation_data=(X_test_supply, y_test_supply), epochs=50, batch_size=32, verbose=1, callbacks=[early_stop])

# Predict over the full range (2024–2029)
predicted_demand = model_demand.predict(X_demand)
predicted_supply = model_supply.predict(X_supply)

# Inverse transform to get actual values
predicted_demand = scaler_demand.inverse_transform(predicted_demand)
predicted_supply = scaler_supply.inverse_transform(predicted_supply)

# Prepare full actual data for comparison
y_demand_actual = scaler_demand.inverse_transform(y_demand.reshape(-1, 1))
y_supply_actual = scaler_supply.inverse_transform(y_supply.reshape(-1, 1))

# Ensure the full date range from 2024-2029 is covered
full_range_index = data.index[time_step:]

# Generate date range for x-ticks (force a full date range to appear)
date_range = pd.date_range(start='2024-01-01', end='2029-12-01', freq='YS')

# Plot Actual vs Predicted for Supply
plt.figure(figsize=(11, 8))
plt.plot(full_range_index, y_supply_actual, label='Actual Supply', color='blue')
plt.plot(full_range_index, predicted_supply, label='Predicted Supply', color='orange', linestyle='--')
plt.title('LSTM: Actual vs Predicted Supply', fontsize=25)
plt.xlabel('Date', fontsize=32)
plt.ylabel('Supply', fontsize=32)
plt.legend(fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.grid(True)

# Explicitly set the x-tick values
plt.gca().xaxis.set_major_locator(mdates.YearLocator())  # Set major ticks at year intervals
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))  # Format x-axis as years
plt.xticks(date_range, date_range.year, fontsize=20)
plt.yticks(fontsize=20)

# Save the plot
plt.savefig(f'{plots_folder}LSTM_supply_actual_vs_predicted_full.pdf', format='pdf')
plt.close()

# Plot Actual vs Predicted for Demand
plt.figure(figsize=(11, 8))
plt.plot(full_range_index, y_demand_actual, label='Actual Demand', color='blue')
plt.plot(full_range_index, predicted_demand, label='Predicted Demand', color='orange', linestyle='--')
plt.title('LSTM: Actual vs Predicted Demand', fontsize=25)
plt.xlabel('Date', fontsize=32)
plt.ylabel('Demand', fontsize=32)
plt.legend(fontsize=20)
plt.grid(True)

# Explicitly set the x-tick values
plt.gca().xaxis.set_major_locator(mdates.YearLocator())  # Set major ticks at year intervals
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))  # Format x-axis as years
plt.xticks(date_range, date_range.year, fontsize=20)
plt.yticks(fontsize=20)

# Save the plot
plt.savefig(f'{plots_folder}LSTM_demand_actual_vs_predicted_full.pdf', format='pdf')
plt.close()

# Plot residuals for Supply
supply_residuals = y_supply_actual - predicted_supply
plt.figure(figsize=(11, 8))
plt.plot(full_range_index, supply_residuals, label='Residuals', color='purple')
plt.title('LSTM: Residuals Plot for Supply', fontsize=25)
plt.xlabel('Date', fontsize=32)
plt.ylabel('Residuals', fontsize=32)
plt.grid(True)

# Explicitly set the x-tick values
plt.gca().xaxis.set_major_locator(mdates.YearLocator())  # Set major ticks at year intervals
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))  # Format x-axis as years
plt.xticks(date_range, date_range.year, fontsize=20)
plt.yticks(fontsize=20)

# Save the plot
plt.savefig(f'{plots_folder}LSTM_supply_residuals_full.pdf', format='pdf')
plt.close()

# Plot residuals for Demand
demand_residuals = y_demand_actual - predicted_demand
plt.figure(figsize=(11, 8))
plt.plot(full_range_index, demand_residuals, label='Residuals', color='purple')
plt.title('LSTM: Residuals Plot for Demand', fontsize=25)
plt.xlabel('Date', fontsize=32)
plt.ylabel('Residuals', fontsize=32)
plt.grid(True)

# Explicitly set the x-tick values
plt.gca().xaxis.set_major_locator(mdates.YearLocator())  # Set major ticks at year intervals
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))  # Format x-axis as years
plt.xticks(date_range, date_range.year, fontsize=20)
plt.yticks(fontsize=20)

# Save the plot
plt.savefig(f'{plots_folder}LSTM_demand_residuals_full.pdf', format='pdf')
plt.close()
